# 0. The Obligatory Part

In [1]:
!pip3 install ortools prettytable gurobipy plotly requests

In [2]:
import pandas as pd
import ortools
from ortools.sat.python import cp_model
import gurobipy as gp
from gurobipy import GRB

# 1. Define The Data Structures

In [3]:
paths = '/content/drive/MyDrive/Tadika Mesra Bunga Matahari/#1 Optimization Problem/burrito-game/'
round = 'muafi_r1d1_2/round1-day1'

file_path = {
    'problem-data': paths + round + "_problem_data.csv",
    'truck-node-data': paths + round + "_truck_node_data.csv",
    'demand-node-data': paths + round + "_demand_node_data.csv",
    'demand-truck-data': paths + round + "_demand_truck_data.csv" ,
}

### Pre-processing: Problem Data

In [4]:
path = file_path['problem-data']
problem_df = pd.read_csv(path)

burrito_price = float(problem_df['burrito_price'][0])       # r
ingredient_cost = float(problem_df['ingredient_cost'][0])   # k
truck_cost = float(problem_df['truck_cost'][0])             # f_j

print(f"The burritos cost ₲{ingredient_cost} to make and are sold for ₲{burrito_price}. Each truck costs ₲{truck_cost} to use per day.")

The burritos cost ₲5.0 to make and are sold for ₲10.0. Each truck costs ₲250.0 to use per day.


### Pre-processing: Truck Node Data

In [5]:
path = file_path['truck-node-data']
truck_node_df = pd.read_csv(path)

truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for i, row in truck_node_df.iterrows()}
truck_spots = truck_coordinates.keys()

print(f"There are {len(truck_spots)} available 'truck_spots' or places where a truck can be placed around Burritoville.")

There are 17 available 'truck_spots' or places where a truck can be placed around Burritoville.


### Pre-processing: Demand Node Data

In [6]:
path = file_path['demand-node-data']
demand_node_df = pd.read_csv(path)

buildings = {}
building_names = {}
building_coordinates = {}
demand = {}

for i, row in demand_node_df.iterrows():
    index = row['index']

    buildings[index] = [row['name'], (float(row['x']), float(row['y'])), float(row['demand'])]
    building_names[index] = row['name']
    building_coordinates[index] = (float(row['x']), float(row['y']))
    demand[index] = float(row['demand'])

print(f"There are in {len(buildings)} buildings with hungry customers also known as demand nodes.")

There are in 14 buildings with hungry customers also known as demand nodes.


### Pre-processing: Demand Truck Data

In [7]:
path = file_path['demand-truck-data']
demand_truck_df = pd.read_csv(path)

building_truck_spot_pairs = {}
distance = {}
scaled_demand = {}

for i, row in demand_truck_df.iterrows():
  demand_node_index = row['demand_node_index']
  truck_node_index = row['truck_node_index']

  if float(row['scaled_demand']) > 0:
    pair = (demand_node_index, truck_node_index)
    distance[pair] = float(row['distance'])
    scaled_demand[pair] = float(row['scaled_demand'])
    building_truck_spot_pairs[pair] = [distance[pair], scaled_demand[pair]]

print(f"There are in {len(building_truck_spot_pairs)} pairs of trucks spots and buildings with hungry customers.")

There are in 113 pairs of trucks spots and buildings with hungry customers.


In [8]:
# Plot the truck spots and customer demands on the Burritoville map
import requests
r = requests.get('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/burrito_optimization_game/util/show_map.py')
with open('show_map_local.py', 'w') as f:
    f.write(r.text)
from show_map_local import show_map

show_map(buildings, building_names, building_coordinates, demand, truck_coordinates)

# 2. Construct the Model

we have two sets

$$
\begin{align*}
\text{sets}
&= \begin{cases}
\mathcal{I}, \text{ set of building with customer demands } i & \quad \texttt{buildings} \\
\mathcal{J}, \text{ set of available truck spots } j & \quad \texttt{truck spots} \\
\end{cases}
\\
\end{align*}
\\
$$

then we have parameters, scalars, and data structures. so let:

$$
\begin{align*}
i & \quad \text{customer demand} \\
j & \quad \text{truck spots} \\
r & \quad \text{revenue per burrito.}  & \quad \texttt{burrito}\_ \texttt{price} \\
k & \quad \text{ingredient cost per burrito} & \quad \texttt{ingredient}\_ \texttt{cost} \\
\alpha_{ij},d_i & \quad \text{demand multiplier and demand. These have been combined into one scaled demand.} & \quad \texttt{scaled}\_ \texttt{demand[i,j]} \\
f_j & \quad \text{cost to place a truck for the day} & \quad \texttt{truck}\_ \texttt{cost}\\
\end{align*}
\\
$$

and the following decision variables:

$$
\begin{align*}
x_j &= \begin{cases}
1, & \text{truck is placed at truck spots } j \in \mathcal{J} \\
0, & \text{otherwise}
\end{cases} & \quad \texttt{x}\_ \texttt{placed[j]} \\
\\
y_{ij} &= \begin{cases}
1, & \text{truck } j \in \mathcal{J} \text{ serves burritos to customer from building } i \in \mathcal{I} \\
0, & \text{otherwise}
\end{cases} & \quad \texttt{y}\_ \texttt{served[i,j]}
\end{align*}
\\ \\
$$


### **Objective Function**

$$
\begin{align*}
{\rm maximize} & \quad \displaystyle \sum_{i\in \mathcal{I}}   \  \displaystyle \sum_{j\in \mathcal{J}}  \  ( r - k) \alpha_{ij} d_i  y_{ij} - \displaystyle \sum_{j\in \mathcal{J}} f_j  x_j \\ \\
{\rm s.t.} & \quad \displaystyle \sum_{j\in \mathcal{J}} y_{ij} \leq 1 & \quad \forall i\in {\mathcal{I}}  \\
& \quad y_{ij} \leq x_j & \quad \forall i\in {\mathcal{I}} ,j\in {\mathcal{J}}  \\
& \quad x_j, y_{ij} \in \{0,1\} &  \quad \forall i\in {\mathcal{I}} ,j\in {\mathcal{J}}  \\ \\
\end{align*}
$$

In [9]:
# construct model
model = cp_model.CpModel()

# 3. Build the Decision Variables

In [10]:
# Create decision variables for x_placed
x_placed = {}
for j in truck_spots:
    x_placed[j] = model.NewBoolVar(f"x-placed_{j}")

# Create decision variables for y_served
y_served = {}

for i, j in building_truck_spot_pairs:
    y_served[i, j] = model.NewBoolVar(f"y-served_{i}_{j}")

In [11]:
print(y_served)

{('demand11', 'truck8'): y-served_demand11_truck8(0..1), ('demand11', 'truck9'): y-served_demand11_truck9(0..1), ('demand11', 'truck12'): y-served_demand11_truck12(0..1), ('demand11', 'truck13'): y-served_demand11_truck13(0..1), ('demand11', 'truck14'): y-served_demand11_truck14(0..1), ('demand11', 'truck15'): y-served_demand11_truck15(0..1), ('demand11', 'truck16'): y-served_demand11_truck16(0..1), ('demand11', 'truck17'): y-served_demand11_truck17(0..1), ('demand11', 'truck27'): y-served_demand11_truck27(0..1), ('demand11', 'truck47'): y-served_demand11_truck47(0..1), ('demand12', 'truck6'): y-served_demand12_truck6(0..1), ('demand12', 'truck8'): y-served_demand12_truck8(0..1), ('demand12', 'truck9'): y-served_demand12_truck9(0..1), ('demand12', 'truck12'): y-served_demand12_truck12(0..1), ('demand12', 'truck13'): y-served_demand12_truck13(0..1), ('demand12', 'truck14'): y-served_demand12_truck14(0..1), ('demand12', 'truck15'): y-served_demand12_truck15(0..1), ('demand12', 'truck16')

# 4. Subject to the Constraints


for the first constraints, that is

$$
\begin{align*}
\displaystyle \sum_{j\in \mathcal{J}} y_{ij} \leq 1 & \quad \quad \forall i\in {\mathcal{I}}  \\
\end{align*}
$$

In [12]:
# Ensure one truck per customers at given building constraint
for i in buildings:
  model.Add(sum(y_served[i, j] for j in truck_spots if (i, j) in y_served) <= 1)

then for the second constraint, given that:

$$
\begin{align*}
y_{bt} \leq x_t & \quad \quad \forall b\in {\mathcal{B}} ,t\in {\mathcal{T}}  \\
\end{align*}
$$

In [13]:
for i, j in building_truck_spot_pairs:
    model.Add(y_served[(i, j)] <= x_placed[j])

# 5. Set The Objective Model

In [14]:
burrito_revenue = sum((burrito_price - ingredient_cost) * scaled_demand[key] * y_served[key] for key in building_truck_spot_pairs)
total_truck_cost = sum(truck_cost * x_placed[j] for j in truck_spots)

model.Maximize(burrito_revenue - total_truck_cost)

# 6. Solve the Model

In [15]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [16]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution found!')

  print(f'Profit: {solver.ObjectiveValue()}')

  for j in truck_spots:
    if solver.BooleanValue(x_placed[j]):
      print(f'Truck located at {j}')

  for i, j in building_truck_spot_pairs:
    if solver.BooleanValue(y_served[i, j]):
      print(f'Customer from building {i} serves by truck at {j}')
else:
  print("No solution found")

Solution found!
Profit: 970.0
Truck located at truck15
Truck located at truck27
Truck located at truck55
Customer from building demand11 serves by truck at truck15
Customer from building demand12 serves by truck at truck15
Customer from building demand18 serves by truck at truck15
Customer from building demand19 serves by truck at truck15
Customer from building demand20 serves by truck at truck15
Customer from building demand21 serves by truck at truck15
Customer from building demand27 serves by truck at truck27
Customer from building demand28 serves by truck at truck27
Customer from building demand29 serves by truck at truck27
Customer from building demand34 serves by truck at truck55
Customer from building demand35 serves by truck at truck55
Customer from building demand40 serves by truck at truck55
Customer from building demand41 serves by truck at truck27
Customer from building demand52 serves by truck at truck15


In [17]:
placed_trucks = [j for j in x_placed if solver.Value(x_placed[j]) == 1]
show_map(buildings, building_names, building_coordinates, demand, truck_coordinates, placed_trucks = placed_trucks)